In [7]:
from joblib import load
import torch
import zipfile
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from models import ECG_CNN_TransformerBIN, ECG_CNN_Transformer
from utils import inference_ovBIN, inference_ov, sigmoid
from sklearn.preprocessing import StandardScaler
import numpy as np
from joblib import load
import pathlib
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import copy

In [10]:
scaler = load('models/scaler_main.joblib')

checkpointBIN = torch.load('models/BinMODELwithNOISEFULLSETT01loss.pt')
modelBIN  = ECG_CNN_TransformerBIN().cuda()
modelBIN.load_state_dict(checkpointBIN)

checkpoint = torch.load('models/WITHNOISEFULLnewf10-56loss0-15classes6.pt')
model  = ECG_CNN_Transformer().cuda()
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [11]:
dsMetatest = pd.read_csv('public_data/test/test/test_meta.csv')
dsMetatest

,patient_id,age,sex,height,weight,record_name
0,11926.0,22.0,1,NaN,52.0,00127_hr
1,12396.0,60.0,1,NaN,NaN,00186_hr
2,17536.0,28.0,0,NaN,112.0,00193_hr
3,21074.0,19.0,1,NaN,59.0,00198_hr
4,18690.0,85.0,1,NaN,NaN,00211_hr
...,...,...,...,...,...,...
444,8951.0,44.0,0,NaN,NaN,21269_hr
445,21392.0,71.0,0,NaN,NaN,21301_hr
446,17212.0,62.0,1,NaN,NaN,21366_hr
447,13332.0,40.0,0,NaN,NaN,21547_hr


In [13]:
record_names = list(dsMetatest['record_name'])
y_train_secondtest = copy(dsMetatest)
dsMetatest.set_index('record_name', inplace=True)

In [15]:
test_X = []
test_meta = []

for key in record_names:
  test_X.append(np.load('public_data/test/test/'+key+'.npy'))
  test_meta.append(dsMetatest.loc[key])

In [16]:
normalized_data_listtest = [scaler.transform(data) for data in test_X]

test_X = normalized_data_listtest

In [17]:
dsMetaTest = pd.read_csv('public_data/test/test/test_meta.csv')
train_record_names = dsMetaTest['record_name']

In [18]:
eval_res = []
for x in range(len(test_X)):
   eval_res.append(inference_ovBIN(modelBIN, torch.from_numpy(test_X[x]).cuda()))
eval_res = [sigmoid(i) for i in eval_res]

In [19]:
reses = [1 if float(j) >= 0.275 else 0 for j in list(eval_res)]

In [20]:
answers = []
for i in range(len(y_train_secondtest)):
  test_record = np.load('public_data/test/test/'+record_names[i]+'.npy')
  test_record = scaler.transform(test_record)

  answers.append(inference_ov(model, torch.from_numpy(test_record).cuda()))

In [25]:
sample = pd.read_csv('public_data/test/sample_submission.csv')

In [27]:
new_arr = []
for k in range(len(answers)):
    for j in range(len(answers[k])):
        new_arr.append(answers[k][j].detach().cpu().numpy())
        
for i in range(len(new_arr)):
    for j in range(len(new_arr[i])):
        try:
            new_arr[i][j] = sigmoid(new_arr[i][j])
        except:
            new_arr[i][j] = 0

ress = []
for kk in range(len(reses)):
    if reses[kk] == 0:
        ress.append([0,0,0,0,0,0,1])
    else:
###########использование максимального элемента вместо трешхолда [0.1, 0.2, 0.3] -> [0, 0, 1] не привело к хорошим результатам
#         tmp_ress = [1 if float(j) >= 0.1 else 0 for j in new_arr[kk]]+[0]
#         count_zero = 0
#         for ll in tmp_ress:
#             if ll == 0:
#                 count_zero += 1            
#         if count_zero == 7:                
#             print(new_arr[kk].shape)
#             zero_mat = np.zeros(new_arr[kk].shape[0] + 1)
#             zero_mat[np.argmax(new_arr[kk], axis = 0)] = 1
# #             print(zero_mat)
# #             print(new_arr[kk])
# #             print(kk)
# #             break
#             ress.append(zero_mat.astype(int))
#             print(kk)
#         else:
        ress.append([1 if float(j) >= 0.1 else 0 for j in new_arr[kk]]+[0])
    
ans = pd.DataFrame(ress, columns = list(sample.columns[1:]))

In [28]:
ans['record_name'] = record_names
ans.set_index('record_name', inplace = True)

In [29]:
for i in range(len(sample)):
    sample.loc[i, 'перегородочный'] = ans.loc[sample.iloc[i]['record_name']]['перегородочный'] 
    sample.loc[i, 'передний'] = ans.loc[sample.iloc[i]['record_name']]['передний']
    sample.loc[i, 'боковой'] = ans.loc[sample.iloc[i]['record_name']]['боковой'] 
    sample.loc[i, 'передне-боковой'] = ans.loc[sample.iloc[i]['record_name']]['передне-боковой']
    sample.loc[i, 'передне-перегородочный'] = ans.loc[sample.iloc[i]['record_name']]['передне-перегородочный'] 
    sample.loc[i, 'нижний'] = ans.loc[sample.iloc[i]['record_name']]['нижний']
    sample.loc[i, 'норма'] = ans.loc[sample.iloc[i]['record_name']]['норма']

In [30]:
sample.to_csv('submit.csv', index = False)